In [1]:
import sys
sys.path.append("..") ## resetting the path to the parent directory
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

keras = tf.keras
AUTOTUNE = tf.data.experimental.AUTOTUNE
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from tensorflow.keras import backend as K

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [33]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [47]:
from tensorflow.keras import callbacks

In [2]:
df = pd.read_csv('../data/coin_metadata.csv')
constantine_df = df[df.portrait=='Antoninus Pius']
augustus_df = df[df.portrait=='Gallienus']
frames = [constantine_df,augustus_df]

demo_df = pd.concat(frames).reset_index(drop=True)
demo_df.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
LE = LabelEncoder()
demo_df['code'] = LE.fit_transform(demo_df['portrait'])

In [4]:
X = demo_df.drop('code',axis=1)

y = demo_df.code

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.2, stratify=y_train)

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
file_path = f'../img/'
train_filenames = [file_path + f'{fname}.jpg' for fname in X_train.fname]
val_filenames = [file_path + f'{fname}.jpg' for fname in X_val.fname]
test_filenames = [file_path + f'{fname}.jpg' for fname in X_test.fname]

In [18]:
IMG_WIDTH=150
IMG_HEIGHT=150
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

In [1]:
def reset_indices(arr):
    arr.reset_index(drop=True,inplace=True)
    return arr

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [150, 150])
    image /= 255.0
    return image

def remove_corrupt_files(filenames,y_arr):
    imgs = []
    y_arr = reset_indices(y_arr)
    for i,filename in enumerate(filenames):        
        # Try reading, decoding, resizing and normalizing images
        # Only appends files that do not cause any errors to an 'imgs' list
        try:
            img = load_and_preprocess_image(filename)
            imgs.append(np.asarray(img))      
        # If error occurs, skip to the next file and remove the file from the y array using its index
        except:
            
            del y_arr[i]
            continue
              
    X = np.asarray(imgs)
    y = np.asarray(y_arr)
    
    return X, y    

In [2]:
X_train,y_train = remove_corrupt_files(train_filenames,y_train)
X_val,y_val = remove_corrupt_files(val_filenames,y_val)
X_test,y_test = remove_corrupt_files(test_filenames,y_test)

NameError: name 'train_filenames' is not defined

In [26]:
train_generator = train_datagen.flow(X_train, y_train,batch_size=30)
val_generator = val_datagen.flow(X_val, y_val, batch_size=30)

In [31]:
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(150,150,3))
restnet.trainable = False

#inpt = Input(shape=(224,224,3),name = 'image_input')
#output = restnet(inpt)
output = restnet.layers[-1].output
output = Flatten()(output)
restnet_model = Model(restnet.input,outputs=output)
#restnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [36]:
final_model = Sequential()
final_model.add(restnet_model)
final_model.add(Dense(512, activation='relu', input_dim=(150,150,3)))
final_model.add(Dropout(0.5))
final_model.add(Dense(512, activation='relu'))
final_model.add(Dropout(0.3))
final_model.add(Dense(2, activation='sigmoid'))

In [45]:
final_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

In [ ]:
epochs = 10
checkpoint_filepath = '../checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [ ]:
my_callbacks = [
    callbacks.EarlyStopping(monitor='loss', patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath='resnet_model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.LearningRateScheduler(schedule, verbose=0)
]

In [46]:
resnet_history = final_model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val,y_val))

Epoch 1/10
69/69 [==============================] - 432s 6s/step - loss: 0.6992 - accuracy: 0.5059 - val_loss: 0.6932 - val_accuracy: 0.4964
Epoch 2/10
69/69 [==============================] - 487s 7s/step - loss: 0.6933 - accuracy: 0.5036 - val_loss: 0.6931 - val_accuracy: 0.5036
Epoch 3/10
69/69 [==============================] - 379s 5s/step - loss: 0.6933 - accuracy: 0.5123 - val_loss: 0.6931 - val_accuracy: 0.4964
Epoch 4/10
69/69 [==============================] - 425s 6s/step - loss: 0.6934 - accuracy: 0.4768 - val_loss: 0.6931 - val_accuracy: 0.5036
Epoch 5/10
69/69 [==============================] - 371s 5s/step - loss: 0.6933 - accuracy: 0.4959 - val_loss: 0.6931 - val_accuracy: 0.5036
Epoch 6/10
69/69 [==============================] - 317s 5s/step - loss: 0.6932 - accuracy: 0.5150 - val_loss: 0.6931 - val_accuracy: 0.4964
Epoch 7/10
69/69 [==============================] - 424s 6s/step - loss: 0.6931 - accuracy: 0.4941 - val_loss: 0.6931 - val_accuracy: 0.4964
Epoch 8/10
15

KeyboardInterrupt: 